In [ ]:
import torch
from det3d.torchie import Config

import pickle

config = "/workspace/centerformer/configs/nusc/nuscenes_centerformer_poolformer.py"

cfg = Config.fromfile(config)
FINDCENTER_GEN_ONNX = True
POOLFORMER_GEN_ONNX = False

In [ ]:
from det3d.models import build_detector
from det3d.torchie.trainer import load_checkpoint

checkpoint_path = "/workspace/centerformer/work_dirs/nuscenes_poolformer/poolformer.pth"

model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
checkpoint = load_checkpoint(model, checkpoint_path, map_location="cpu");

In [ ]:
from torch import nn

class CenterFinder(nn.Module):
    def __init__(self, model):
        super(CenterFinder, self).__init__()
        self.model = model

    def forward(self, x):
        # x = self.model.neck(x)
        # preds = self.model.bbox_head(x)
        # return preds
        return self.model.neck.find_centers(x) # pos_features, out_dict_list, ct_feat 
    
centerFinder = CenterFinder(model)
# centerFinder.cuda()
centerFinder.eval();

In [ ]:
import pickle
# with open("/workspace/centerformer/work_dirs/partition/sample_data/x.pkl", 'wb') as handle:
#     pickle.dump(x, handle)
# with open("/workspace/centerformer/work_dirs/partition/sample_data/example.pkl", 'wb') as handle:
#     pickle.dump(example, handle)

pickle_dir = "/workspace/centerformer/work_dirs/partition/sample_data/"

with open(pickle_dir + "findcenter_input.pkl", 'rb') as handle:
    x = pickle.load(handle)
# with open(pickle_dir + "example.pkl", 'rb') as handle:
#     example = pickle.load(handle)

In [ ]:
with torch.no_grad():
    ct_feat, center_pos_embedding, out_scores, out_labels, out_orders, out_masks = centerFinder(x)
    
with open(pickle_dir + "findcenter_output1.pkl", 'rb') as handle:
    output1 = pickle.load(handle)
with open(pickle_dir + "findcenter_output2.pkl", 'rb') as handle:
    output2 = pickle.load(handle)
with open(pickle_dir + "findcenter_output3.pkl", 'rb') as handle:
    output3 = pickle.load(handle)
    
print((output1 == ct_feat).all())
print((output2 == center_pos_embedding).all())
print(ct_feat[0][0][:10])
print(output1[0][0][:10])

In [ ]:
import onnx
import numpy as np

if FINDCENTER_GEN_ONNX:
    model_name = "findCenter.onnx"
    # dummy_input=torch.randn(x.shape, requires_grad=True).cuda()
    dummy_input = torch.as_tensor(np.ones([1, 256, 180, 180]), dtype=torch.float32)
    torch.onnx.export(centerFinder, (dummy_input), model_name,
                input_names=['input_tensor'], 
                output_names=['ct_feat', 'center_pos_embedding', 'out_scores', 'out_labels', 'out_orders', 'out_masks'],
                export_params=True, 
                do_constant_folding=True,
                opset_version=11,
                )
    
    # !export POLYGRAPHY_AUTOINSTALL_DEPS=1
    !polygraphy surgeon sanitize findCenter.onnx --fold-constants --output findCenter_folded.onnx

    onnx.checker.check_model(onnx.load("findCenter_folded.onnx"))
    print("gen findCenter.onnx success!")
else:
    print("pass")

In [ ]:
dummy_input.shape

In [ ]:
with torch.no_grad():
    ct_feat = centerFinder.model.neck.poolformer_forward(ct_feat, center_pos_embedding)
            
out_dict_list = []
num_tasks = len(centerFinder.model.neck.tasks)
obj_num = centerFinder.model.neck.obj_num
for idx in range(num_tasks):
    out_dict = {}
    out_dict.update(
        {
            "scores": out_scores[idx],
            "labels": out_labels[idx],
            "order": out_orders[idx],
            "mask": out_masks[idx],
            "ct_feat": ct_feat[:, :, idx * obj_num : (idx+1) * obj_num],
        }
    )
    out_dict_list.append(out_dict)

In [ ]:
with open(pickle_dir + "neck_output.pkl", 'rb') as handle:
    output_poolformer = pickle.load(handle)

In [ ]:
out_dict_list[0].keys()

In [ ]:
for i in range(len(out_dict_list)):
    print(i)
    print('scores', (output_poolformer[i]['scores']==out_dict_list[i]['scores']).all())
    print('labels', (output_poolformer[i]['labels']==out_dict_list[i]['labels']).all())
    print('orders', (output_poolformer[i]['order']==out_dict_list[i]['order']).all())
    print('masks', (output_poolformer[i]['mask']==out_dict_list[i]['mask']).all())
    print('ct_feat', (output_poolformer[i]['ct_feat']==out_dict_list[i]['ct_feat']).all())

In [ ]:
(output_poolformer[i]['mask']==out_dict_list[i]['mask'])